In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
import pickle
import aws
from parsl import *
import parsl
provider = aws.EC2Provider('providerconf.json')


In [ ]:
import time
import numpy as np
from ipyparallel import Client
rc=Client()
executor = parsl.executors.ipp.IPyParallelExecutor()
dfk = DataFlowKernel(executor)

In [ ]:
for i in range(20):
    time.sleep(10)
    provider.scale_out(1)
    start=time.time()
    while True:
        try:
            view = rc[:]
            end = time.time()
            times.append(end-start)
            provider.scale_in(1)
            break
        except Exception as e:
            pass
        

In [ ]:
times=[111,111,107.,104.26447439193726, 114.74069857597351,114.83456110954285,112.26447439193726, 118.74069857597351,113.83456110954285,119.26447439193726, 121.74069857597351,123.83456110954285,116.26447439193726, 108.74069857597351,123.83456110954285, 114.56703305244446, 117.43648338317871, 114.235]
print(len(times))
plt.hist(times, normed=True, bins=10, color='green')
plt.title("Distribution of spin-up times")
plt.xlabel("Time until engine registration (sec)")
plt.ylabel("Relative frequency (total area: 1)")

In [ ]:
@App('python', dfk)
def get_num(first, second):
    return first + second 

def test_fibonacci(num = 5):
    x1 = 0
    x2 = 1
    counter = 0
    results = [ ]
    results.append(0)
    results.append(1)
    while counter < num - 2:
        counter += 1 
        results.append(get_num(x1, x2))
        temp = x2
        x2 = get_num(x1, x2)
        x1 = temp
        results[-1].result()
    for i in range(len(results)):
        if type(results[i]) is int:
            #print(results[i])
            pass
        else:
            pass

In [ ]:
@App('python',dfk)
def double(x):
    return x*2


def mapReduce(n):
    tic=time.time()
    jobs = []
    for i in range(0,n):
        jobs.extend([double(i)])
    #result = sum(res.result() for res in jobs)
    jobs[-1].result()
    toc = time.time()
    return toc-tic

In [ ]:
mapReduce(500)

In [ ]:
#1 node 11.3sec #2engines: 9.254sec #3engines 11.4sec, 4engines 10.6sec
tic = time.time()
test_fibonacci(900)
tac = time.time()
diff = tac-tic
diff

In [ ]:
@App('python', dfk)
def parsl_test_app(sleeptime, in_data, out_data_size=0):
    import time
    import numpy as np
    import os
    time.sleep(sleeptime)
    return np.random.rand(int(out_data_size))

In [ ]:
def test_latency(n):
    fus = []
    tic = time.time()
    echo = lambda x: x
    tic = time.time()
    for i in range(n):
        fus.append(parsl_test_app(0,[5],55))
    toc = time.time()
    fus[-1].result()
    tac = time.time()
    rt = tac-tic
    sent = toc-tic
    return sent, rt

In [ ]:
#Task throughput as a function of nodes
# 1, 2, 4, 8, 16 t2.small. Do c4.large
#provider.scale_in(10)
allRates = []
clusterOptions = [1,2,4,8,16]
for i, size in enumerate(clusterOptions):
    rates = []
    increase = (size - clusterOptions[i-1]) if i > 0 else 1
    print(increase)
    #provider.scale_out(increase)
    for n in [300,300,300,300,300,300]:
        # short rest between tests
        time.sleep(5)
        s, rt = test_latency(n)
        rates.append(n/s)
        print( "%4i %6.1f %6.1f" % (n,n/s, n/rt))
    allRates.append(np.mean(rates))

In [ ]:
plt.plot(clusterOptions, allRates)
plt.plot(clusterOptions, allRates)
plt.xlabel("nodes")
plt.ylabel("tasks/sec")
plt.plot()
plt.yticks(range(500,2000,100))
provider.scale_in(16)
plt.show()

In [ ]:
provider.scale_out(1)

In [ ]:
#Task throughput as a function of nodes
# 1, 2, 4, 8, 16 t2.small. Do c4.large
rates1 = []
tasks1 = []
for n in [100,200,300,400,500,600,700,800,900,1000,2000,5000]:
    # short rest between tests
    time.sleep(5)
    s = test_latency(n)
    rates.append(n/s)
    tasks.append(n)
    print( "%4i %6.1f %6.1f" % (n,n/s, n/s))

In [ ]:
plt.plot(tasks1, rates1)
plt.title("Job submit rate vs number of jobs (1 node) with C4.large instances")
plt.xlabel("jobs")
plt.ylabel("tasks/sec")
plt.show()

In [ ]:
def test_throughput(n):
    fus=[]
    tic = time.time()
    for i in range(n):
        a = parsl_test_app(0.8,5,0)
        fus.append(a)
    toc = time.time()  
    fus[-1].result()
    tac = time.time()
    sent = toc-tic
    roundtrip = tac-tic
    return sent, roundtrip

In [ ]:
provider.scale_in(1)

In [ ]:
allRates1 = []
clusterOptions = [1,2,4,8,16]
for i, size in enumerate(clusterOptions):
    increase = (size - clusterOptions[i-1]) if i > 0 else 1
    print(increase)
    #provider.scale_out(increase)
    #time.sleep(300)
    for n in [100,200,300,400,500,600,700,800,900,1000]:
        rates = []
        # short rest between tests
        time.sleep(5)
        s,rt = test_throughput(n)
        rates.append(n/rt)
        print( "%4i %6.1f %6.1f" % (n,n/s, n/rt))
    allRates1.append(np.mean(rates))

In [ ]:
plt.plot(clusterOptions, allRates1)
plt.title("Job throughput (completion) vs number of nodes with C4.large instances (1 ms jobs)")
plt.xlabel("nodes")
plt.ylabel("tasks/sec")
plt.show()

In [ ]:
def test_map(v,dt,n):
    ts = [dt]*n
    tic = time.time()
    amr = v.map_async(time.sleep, ts)
    toc = time.time()
    amr.get()
    tac = time.time()
    sent = toc-tic
    roundtrip = tac-tic
    return sent, roundtrip

In [ ]:
view = rc.load_balanced_view()
n = len(rc.ids) * 16
sizes = []
rates = []
for dt in np.logspace(-3,0,7):
    time.sleep(0.5)
    s,rt = test_map(view, dt, n)
    print("%4ims %5.1f%%" % (1000*dt, 1600*dt / rt))
    sizes.append(1000*dt)
    rates.append(1600*dt / rt)
plt.plot(sizes, rates)
plt.plot(sizes, rates)
plt.title("CPU Utilization vs Task Completion Time (16 AWS c4.large nodes)")
plt.xlabel('Task Size (ms to completion)')
plt.ylabel('Utilization')

In [ ]:
from scipy.integrate import quad
from numpy import exp

In [ ]:
def integrand(x):
    return exp(-(x)**2)
print("integrand(1) ={}".format(integrand(1)))

def integral(x):
    return quad(integrand,0,x)[0]
print ("integral(1) ={}".format(integral(1)))

N = 1000
%timeit -n 10 map(integral, range(N))

In [ ]:
pool = rc[:]
print("# CPUs {}".format(len(pool.client.ids)))

In [ ]:
with pool.sync_imports():
    from scipy.integrate import quad
    from numpy import exp
pool['integrand'] = integrand

In [ ]:
print(pool.map_async(integral, range(10)))
%timeit -n 10 pool.map(integral, range(N))

In [ ]:
pool.execute('import numpy')

In [ ]:
# this runs on the engins
def calc_pi_on_engines(n):
    x = numpy.random.rand(n)
    y = numpy.random.rand(n)
    r = numpy.hypot(x, y)
    return 4. * (r < 1.).sum() / n


def calc_pi(n):
    """Estimate pi using IPython.parallel."""
    n_engines = n/len(view)
    results = view.apply_sync(calc_pi_on_engines, n_engines)
    return float(sum(results))/len(results)


def main(N):
    result, time = calc_pi(N)
    print('time  : %3.4g\nresult: %.7f' % (time, result))
    client.purge_everything()


In [ ]:
def test_throughput(v, n, s):
    A = np.random.random(int(s/8)) # doubles are 8B
    tic = time.time()
    echo = lambda x: x
    tic = time.time()
    for i in range(n):
        v.apply_async(echo, A)
    toc = time.time()
    v.wait()
    tac = time.time()
    sent = toc-tic
    roundtrip = tac-tic
    return sent, roundtrip

In [ ]:
view = rc[:]
n = 128
throughput = []
for sz in [1e1,1e2,1e3,1e4,1e5,5e5,1e6,2e6]:
    # short rest between tests
    time.sleep(1)
    s,rt = test_throughput(view, n, int(sz))
    throughput.append(1e-6*sz*n/rt)
    print("%8f  %6.1f t/s  %6.1f t/s %9.3f Mbps" % (sz,n/s,n/rt, 1e-6*sz*n/rt))

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel("Task Size (input data size in bytes)", color="blue")
plt.title("Network Throughput vs Task Input Data Size (4 AWS c4.large nodes)")
ax1.set_xscale('log')
ax1.plot([1e1,1e2,1e3,1e4,1e5,5e5,1e6,2e6], throughput, color='blue')
ax1.set_ylabel("Throughput (Mbps)", color="blue")
plt.plot()

In [ ]:
a=parsl_test_app(5,[5,6,7],55)

In [ ]:
print(a.result())

In [ ]:
provider.scale_out(3)

In [ ]:
view = Client()[:]


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
provider = parsl.execution_provider.aws.EC2Provider('path/to/config')
executor = parsl.executors.ipp.IPyParallelExecutor()
dfk = DataFlowKernel(executor)
@App('python', dfk)
def parsl_app(n):
    arbitrary_function()
    return 0
provider.scale_out(16)
#parsl will automatically run the code on all workers it can find
results = [parsl_app(n).result() for n in range(1,100,1)]
